In [1]:
pip list

Package             Version
------------------- ---------
appnope             0.1.2
argon2-cffi         21.1.0
attrs               21.2.0
backcall            0.2.0
bleach              4.1.0
blis                0.7.5
catalogue           2.0.6
certifi             2021.10.8
cffi                1.15.0
charset-normalizer  2.0.7
click               8.0.3
cymem               2.0.6
debugpy             1.5.1
decorator           5.1.0
defusedxml          0.7.1
en-core-web-sm      3.1.0
entrypoints         0.3
idna                3.3
ipykernel           6.4.2
ipython             7.28.0
ipython-genutils    0.2.0
ipywidgets          7.6.5
jedi                0.18.0
Jinja2              3.0.2
jsonschema          4.1.2
jupyter             1.0.0
jupyter-client      7.0.6
jupyter-console     6.4.0
jupyter-core        4.9.1
jupyterlab-pygments 0.1.2
jupyterlab-widgets  1.0.2
MarkupSafe          2.0.1
matplotlib-inline   0.1.3
mistune             0.8.4
ml-datasets         0.2.0
murmurhash          1.0.6
n

In [2]:
#Importing the libraries

import spacy
from spacy.tokens import DocBin

In [3]:
# Import pandas & read csv file
import pandas as pd
reviews=pd.read_csv("https://raw.githubusercontent.com/hanzhang0420/Women-Clothing-E-commerce/master/Womens%20Clothing%20E-Commerce%20Reviews.csv")

# Extract desired columns and view the dataframe
reviews = reviews[['Review Text','Recommended IND']].dropna()
reviews.head(10)

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
5,"I love tracy reese dresses, but this one is no...",0
6,I aded this in my basket at hte last mintue to...,1
7,"I ordered this in carbon for store pick up, an...",1
8,I love this dress. i usually get an xs but it ...,1
9,"I'm 5""5' and 125 lbs. i ordered the s petite t...",1


In [4]:
# Converting the dataframe into a list of tuples
reviews['tuples'] = reviews.apply(lambda row: (row['Review Text'],row['Recommended IND']), axis=1)
train =reviews['tuples'].tolist()
train[:10]

[('Absolutely wonderful - silky and sexy and comfortable', 1),
 ('Love this dress!  it\'s sooo pretty.  i happened to find it in a store, and i\'m glad i did bc i never would have ordered it online bc it\'s petite.  i bought a petite and am 5\'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.',
  1),
 ('I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c',
  0),
 ("I love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time i wear it, i get nothing b

In [5]:
type(train)

list

In [16]:
len(train)

22641

In [6]:
def make_docs(data):
    docs = []
    for doc, label in nlp.pipe(data, as_tuples=True):
        if label == 0:
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return (docs)

In [ ]:
pip install -U https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.1.0/en_core_web_sm-3.1.0.tar.gz

In [7]:
nlp=spacy.load("en_core_web_sm")

In [9]:
split = 0.8
split = int(len(train) * split)
train_data = train[:split]
valid_data = train[split:]

In [14]:
len(train_data)

18112

In [15]:
len(valid_data)

4529

In [17]:
num_texts = 500

train_docs = make_docs(train_data[:num_texts])
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./data/train.spacy")

valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("./data/valid.spacy")



In [18]:
!python3 -m spacy init fill-config ./base_config.cfg ./config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [19]:
!python3 -m spacy train config.cfg --output ./output

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-10-28 19:46:37,367] [INFO] Set up nlp object from config
[2021-10-28 19:46:37,374] [INFO] Pipeline: ['tok2vec', 'textcat']
[2021-10-28 19:46:37,377] [INFO] Created vocabulary
[2021-10-28 19:46:37,378] [INFO] Finished initializing nlp object
[2021-10-28 19:46:38,220] [INFO] Initialized pipeline components: ['tok2vec', 'textcat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ------------  ----------  ------
  0       0          0.00          0.50       44.44    0.44
  0     200          0.00         61.90       45.48    0.45
  1     400          0.00         28.84       51.99    0.52
  1     600          0.00         13.66       66.69    0.67
  2     800

In [ ]:
import random

def load_data(limit=0, split=0.8):
    train_data=train
    
    # Shuffle the data
    random.shuffle(train_data)
    texts, labels = zip(*train_data)
    
    # Splitting the training and evaluation data
    split = int(len(train_data) * split)
    
    return (texts[:split], labels[:split]), (texts[split:], labels[:split])

n_texts=23486

# Calling the load_data() function
(train_texts, train_labels), (dev_texts, dev_labels) = load_data(limit=n_texts)


# Processing the final format of training data
train_data = list(zip(train_texts,[label: for label in train_labels]))


In [ ]:
type()

In [ ]:
train_data[:10]